# Imports

In [ ]:
!pip install mtcnn
!pip install facenet-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 715.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/1

In [ ]:
import os
import urllib
import numpy as np
import cv2
from PIL import Image

import torch
import timm
from facenet_pytorch import MTCNN

import tensorflow as tf
from tensorflow import keras

import gradio as gr

# Device Setup

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


# Model Load

In [ ]:
# MTCNN for face detection/alignment
mtcnn = MTCNN(image_size=224, margin=20, keep_all=False, device=device)

# ViT backbone (PyTorch)
vit_model = timm.create_model("vit_base_patch16_224", pretrained=True)
vit_model.head = torch.nn.Identity()  # remove classifier head
vit_model.to(device)
vit_model.eval()

# Load Keras classifiers
VIT_classifier = keras.models.load_model("/content/final_vit_classifier.keras")
# RESNET_classifier = keras.models.load_model("/content/RESNET_Classifier.keras")


# # ResNet embedding model (penultimate layer)
# # Assuming last layer is classifier
# RESNET_embedding_model = keras.Model(
#     inputs=RESNET_classifier.input,
#     outputs=RESNET_classifier.layers[-2].output
# )

# Load Image

In [ ]:
def load_image(path_or_url):
    """Loads image from local path or URL"""
    try:
        if path_or_url.startswith("http://") or path_or_url.startswith("https://"):
            resp = urllib.request.urlopen(path_or_url)
            img_array = np.asarray(bytearray(resp.read()), dtype=np.uint8)
            img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        else:
            img = cv2.imread(path_or_url)
        return img
    except Exception as e:
        print("Error loading image:", e)
        return None

# Detect & Crop Face

In [ ]:
def detect_faces(image_bgr):
    """Detect all face bounding boxes using MTCNN"""
    detector = MTCNN(keep_all=True, device=device)
    img_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    boxes, probs = detector.detect(img_rgb)
    if boxes is None:
        return []
    return boxes.astype(int)

In [ ]:
def crop_face(image_bgr, box):
    """Crop a face region from bounding box"""
    x1, y1, x2, y2 = box
    return image_bgr[y1:y2, x1:x2]

# Get Emmbeddings

In [ ]:
# ============================================================
# 7. VIT EMBEDDING
# ============================================================
def get_vit_embedding(face_bgr):
    """
    Takes a cropped face image, aligns it via MTCNN,
    then extracts a ViT embedding (768 dimensions).
    """
    img = Image.fromarray(cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB))
    aligned = mtcnn(img)  # returns a normalized face tensor

    if aligned is None:
        return None

    aligned = aligned.unsqueeze(0).to(device)
    aligned = (aligned - 0.5) / 0.5  # normalization

    with torch.no_grad():
        emb = vit_model(aligned).cpu().numpy().flatten()

    return emb
# ============================================================
# 8. RESNET EMBEDDING
# ============================================================
# def preprocess_resnet(face_bgr):
#     """Preprocess face for ResNet Keras model"""
#     img = cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, (224, 224))
#     img = img / 255.0  # normalize 0-1
#     # Standardization (mean/std same as PyTorch)
#     img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
#     img = np.expand_dims(img, axis=0).astype("float32")
#     return img

# def get_resnet_embedding(face_bgr):
#     """Extract ResNet embedding from penultimate layer"""
#     img = preprocess_resnet(face_bgr)
#     emb = RESNET_embedding_model.predict(img)[0]
#     return emb

# Get Prediction

In [ ]:
def classify_embedding(embedding, model_name="ViT"):
    """Run Keras classifier on embedding"""
    if model_name == "ViT":
        pred = VIT_classifier.predict(np.expand_dims(embedding, axis=0))[0]
    else:
        # pred = RESNET_classifier.predict(np.expand_dims(embedding, axis=0))[0]
        pass

    label_idx = np.argmax(pred)
    confidence = float(pred[label_idx])
    label = "Real" if label_idx == 1 else "Fake"
    return label, confidence

# Draw Bounding Box

In [ ]:
def draw_prediction_box(image, box, label):
    """Draw bounding box + label above the box"""
    x1, y1, x2, y2 = box
    color = (0, 255, 0) if label == "Real" else (0, 0, 255)
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    cv2.putText(
        image,
        label,
        (x1, y1 - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        color,
        2
    )

# Back Pipeline

In [ ]:
def BackEnd_Pipeline(model_name, filetype, file):
    """Gradio backend: file -> detect faces -> embedding -> classify -> draw boxes"""
    if file is None:
        return "❌ Please upload a file.", "", None

    # file is PIL Image
    pil_img = file.convert("RGB")
    img_np = np.array(pil_img)[:, :, ::-1].copy()  # PIL->BGR

    boxes = detect_faces(img_np)
    if len(boxes) == 0:
        return "❌ No faces detected!", "", file

    output_img = img_np.copy()
    results = []

    for box in boxes:
        face = crop_face(img_np, box)

        if model_name == "ViT Model":
            emb = get_vit_embedding(face)
            if emb is None:
                continue
            label, confidence = classify_embedding(emb, "ViT")

        elif model_name == "ResNet Model":
            # emb = get_resnet_embedding(face)
            # label, confidence = classify_embedding(emb, "ResNet")
            pass

        draw_prediction_box(output_img, box, label)
        results.append((label, confidence))

    # Resize output for display
    output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)
    output_img = cv2.resize(output_img, (800, 1000))

    # Return first face result (if multiple faces, can be extended)
    return results[0][0], f"{results[0][1]:.4f}", output_img

# Gradio UI

In [ ]:
with gr.Blocks(title="DeepFake Detector") as demo:
    gr.Markdown(
        """
        # 🔍 DeepFake Detector
        Select a model, choose file type, upload your file, and click **Submit**.
        """
    )

    with gr.Row():
        model_choice = gr.Dropdown(
            ["ViT Model", "ResNet Model"],
            value="ViT Model",
            label="Select Model"
        )
        filetype_choice = gr.Dropdown(
            ["Image"],
            value="Image",
            label="Select File Type"
        )

    with gr.Row():
        img_input = gr.Image(type="pil", label="Upload Image", interactive=True)

    submit_btn = gr.Button("Submit")

    output_label = gr.Textbox(label="Prediction")
    output_conf = gr.Textbox(label="Confidence Score")
    output_image = gr.Image(label="Processed Image")

    submit_btn.click(
        BackEnd_Pipeline,
        inputs=[model_choice, filetype_choice, img_input],
        outputs=[output_label, output_conf, output_image],
        show_progress=True
    )

demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e464a974c6a99db9fa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# ......................................................................

In [ ]:
# ============================================================
# 0. IMPORTS
# ============================================================
import os
import numpy as np
import cv2
from PIL import Image
import torch
from torchvision import models, transforms
from facenet_pytorch import MTCNN
import timm
from tensorflow import keras
import gradio as gr

# ============================================================
# 1. DEVICE SETUP
# ============================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
torch.backends.cudnn.benchmark = True

# ============================================================
# 2. MODELS: Face Detector, ViT & ResNet
# ============================================================
# Global MTCNN instance
mtcnn = MTCNN(image_size=224, margin=20, keep_all=False, device=device)

# ViT Model
vit_model = timm.create_model("vit_base_patch16_224", pretrained=True)
vit_model.head = torch.nn.Identity()
vit_model.to(device)
vit_model.eval()

# ResNet Model for embeddings
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
resnet.fc = torch.nn.Identity()
resnet.to(device)
resnet.eval()

# Preprocess for ResNet
resnet_preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load Keras classifiers
VIT_classifier = keras.models.load_model("/content/final_vit_classifier.keras")
RESNET_classifier = keras.models.load_model("/content/final_resnet_classifier.keras")

# ============================================================
# 3. FACE DETECTION
# ============================================================
def detect_faces(image_bgr):
    img_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    boxes, probs = mtcnn.detect(img_rgb)
    if boxes is None:
        return []
    return boxes.astype(int)

# ============================================================
# 4. FACE CROPPING
# ============================================================
def crop_face(image_bgr, box):
    x1, y1, x2, y2 = box
    return image_bgr[y1:y2, x1:x2]

# ============================================================
# 5. EMBEDDING EXTRACTION
# ============================================================
def get_vit_embedding(face_bgr):
    img = Image.fromarray(cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB))
    img = img.resize((224, 224))

    # Correct ViT preprocessing
    transform_vit = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    img_tensor = transform_vit(img).unsqueeze(0).to(device)

    with torch.no_grad():
        emb = vit_model(img_tensor).cpu().numpy().flatten()

    return emb

def get_resnet_embedding(face_bgr):
    img = Image.fromarray(cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB))
    img_tensor = resnet_preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = resnet(img_tensor).cpu().numpy().flatten()
    return emb

# ============================================================
# 6. CLASSIFICATION
# ============================================================
def classify_embedding(embedding, model_type="ViT"):
    if model_type == "ViT":
        pred = VIT_classifier.predict(embedding.reshape(1, -1))[0][0]
        label = "Fake" if pred >= 0.5 else "Real"
        confidence = float(pred) if label == "Fake" else 1.0 - float(pred)

    else:
        pred = RESNET_classifier.predict(embedding.reshape(1, -1))[0][0]
        label = "Real" if pred >= 0.5 else "Fake"
        confidence = float(pred) if label == "Real" else 1.0 - float(pred)

    return label, confidence

# ============================================================
# 7. DRAW BOUNDING BOX
# ============================================================
def draw_prediction_box(image, box, label):
    x1, y1, x2, y2 = box
    color = (0, 255, 0) if label == "Real" else (0, 0, 255)
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    cv2.putText(
        image,
        label,
        (x1, max(0, y1 - 10)),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        color,
        2,
        cv2.LINE_AA
    )

# ============================================================
# 8. BACKEND PIPELINE FOR GRADIO
# ============================================================
def BackEnd_Pipeline(model, filetype, file):
    if file is None:
        return "❌ Please upload a file.", "", None

    img_bgr = cv2.cvtColor(np.array(file.convert("RGB")), cv2.COLOR_RGB2BGR)
    boxes = detect_faces(img_bgr)
    if len(boxes) == 0:
        return "❌ No face detected!", "", file

    output_img = img_bgr.copy()
    results = []

    for box in boxes:
        face = crop_face(img_bgr, box)

        if model == "ViT Model":
            emb = get_vit_embedding(face)
            if emb is None:
                label, confidence = "❌ No face detected!", 0.0
            else:
                label, confidence = classify_embedding(emb, "ViT")

        elif model == "ResNet Model":
            emb = get_resnet_embedding(face)
            label, confidence = classify_embedding(emb, "ResNet")

        draw_prediction_box(output_img, box, label)
        results.append((label, confidence))

    output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)
    output_img = cv2.resize(output_img, (1000, 800))

    return results[0][0], f"{results[0][1]:.4f}", output_img

# ============================================================
# 9. GRADIO UI
# ============================================================
with gr.Blocks(title="DeepFake Detector") as demo:
    gr.Markdown("""
    # 🔍 DeepFake Detector
    Select a model, choose the file type, upload your file, and click **Submit**.
    """)

    with gr.Row():
        model_choice = gr.Dropdown(
            ["ViT Model", "ResNet Model"],
            value="ViT Model",
            label="Select Model"
        )
        filetype_choice = gr.Dropdown(
            ["Image"],
            value="Image",
            label="Select File Type"
        )

    with gr.Row():
        img_input = gr.Image(type="pil", label="Upload Image", interactive=True)

    submit_btn = gr.Button("Submit")

    output_label = gr.Textbox(label="Prediction")
    output_conf = gr.Textbox(label="Confidence Score")
    output_image = gr.Image(label="Uploaded / Processed Image")

    submit_btn.click(
    BackEnd_Pipeline,
    inputs=[model_choice, filetype_choice, img_input],
    outputs=[output_label, output_conf, output_image]
)
demo.launch(debug=True)

Using device: cpu


ValueError: File format not supported: filepath=/content/final_vit_classifier.keras.zip. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/content/final_vit_classifier.keras.zip, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
# ============================================================
# 0. IMPORTS
# ============================================================
import os
import numpy as np
import cv2
from PIL import Image
import torch
from torchvision import models, transforms
from facenet_pytorch import MTCNN
import timm
from tensorflow import keras
import gradio as gr

# ============================================================
# 1. DEVICE SETUP
# ============================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
torch.backends.cudnn.benchmark = True

# ============================================================
# 2. MODELS: Face Detector, ViT & ResNet
# ============================================================
# Global MTCNN instance
mtcnn = MTCNN(image_size=224, margin=20, keep_all=False, device=device)

# ViT Model
vit_model = timm.create_model("vit_base_patch16_224", pretrained=True)
vit_model.head = torch.nn.Identity()
vit_model.to(device)
vit_model.eval()

# ResNet Model for embeddings
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
resnet.fc = torch.nn.Identity()
resnet.to(device)
resnet.eval()

# Preprocess for ResNet
resnet_preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load Keras classifiers
VIT_classifier = keras.models.load_model("/content/final_vit_classifier.keras")
RESNET_classifier = keras.models.load_model("/content/final_resnet_classifier.keras")

# ============================================================
# 3. FACE DETECTION
# ============================================================
def detect_faces(image_bgr):
    img_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    boxes, probs = mtcnn.detect(img_rgb)
    if boxes is None:
        return []
    return boxes.astype(int)

# ============================================================
# 4. FACE CROPPING
# ============================================================
def crop_face(image_bgr, box):
    x1, y1, x2, y2 = box
    return image_bgr[y1:y2, x1:x2]

# ============================================================
# 5. EMBEDDING EXTRACTION
# ============================================================
def get_vit_embedding(face_bgr):
    img = Image.fromarray(cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB))
    img = img.resize((224, 224))

    # Correct ViT preprocessing
    transform_vit = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    img_tensor = transform_vit(img).unsqueeze(0).to(device)

    with torch.no_grad():
        emb = vit_model(img_tensor).cpu().numpy().flatten()

    return emb

def get_resnet_embedding(face_bgr):
    img = Image.fromarray(cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB))
    img_tensor = resnet_preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = resnet(img_tensor).cpu().numpy().flatten()
    return emb

# ============================================================
# 6. CLASSIFICATION
# ============================================================
def classify_embedding(embedding, model_type="ViT"):
    if model_type == "ViT":
        pred = VIT_classifier.predict(embedding.reshape(1, -1))[0][0]
        label = "Fake" if pred >= 0.5 else "Real"
        confidence = float(pred) if label == "Fake" else 1.0 - float(pred)

    else:
        pred = RESNET_classifier.predict(embedding.reshape(1, -1))[0][0]
        label = "Real" if pred >= 0.5 else "Fake"
        confidence = float(pred) if label == "Real" else 1.0 - float(pred)

    return label, confidence

# ============================================================
# 7. DRAW BOUNDING BOX
# ============================================================
def draw_prediction_box(image, box, label):
    x1, y1, x2, y2 = box
    color = (0, 255, 0) if label == "Real" else (0, 0, 255)
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    cv2.putText(
        image,
        label,
        (x1, max(0, y1 - 10)),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        color,
        2,
        cv2.LINE_AA
    )

# ============================================================
# 8. BACKEND PIPELINE FOR GRADIO
# ============================================================
def BackEnd_Pipeline(model, filetype, file):
    if file is None:
        return "❌ Please upload a file.", "", None

    img_bgr = cv2.cvtColor(np.array(file.convert("RGB")), cv2.COLOR_RGB2BGR)
    boxes = detect_faces(img_bgr)
    if len(boxes) == 0:
        return "❌ No face detected!", "", file

    output_img = img_bgr.copy()
    results = []

    for box in boxes:
        face = crop_face(img_bgr, box)

        if model == "ViT Model":
            emb = get_vit_embedding(face)
            if emb is None:
                label, confidence = "❌ No face detected!", 0.0
            else:
                label, confidence = classify_embedding(emb, "ViT")

        elif model == "ResNet Model":
            emb = get_resnet_embedding(face)
            label, confidence = classify_embedding(emb, "ResNet")

        draw_prediction_box(output_img, box, label)
        results.append((label, confidence))

    output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)
    output_img = cv2.resize(output_img, (1000, 800))

    return results[0][0], f"{results[0][1]:.4f}", output_img
# --------------------------------------------------
# HOME PAGE CONTENT
# --------------------------------------------------
with gr.Blocks(title="Deep Fake Identification") as home_page:
    gr.Markdown(
        """
        # 🧠 Deep Fake Identification
        ### Detecting forged media using advanced deep learning models
        """
    )

    gr.Markdown("---")

    # Model Information Section
    with gr.Row():
        with gr.Column():
            gr.Markdown(
                """
                ### 🔍 ResNet
                - Convolutional Neural Network with residual connections
                - Strong at learning spatial features
                - **F1 Score: ~82%**
                """
            )
        with gr.Column():
            gr.Markdown(
                """
                ### 🤖 Vision Transformer (ViT)
                - Attention-based transformer architecture
                - Captures global facial artifacts effectively
                - **F1 Score: ~89%**
                """
            )

    gr.Markdown("---")

    # Advertisement / Showcase Section
    gr.Markdown(
        """
        ## 📢 Model Showcase
        Upload a **single image containing both real and fake faces**.
        Our model learns subtle visual artifacts to distinguish manipulated regions from authentic ones.
        """
    )

    # Changed to a static image display with the downloaded example image
    showcase_upload = gr.Image(label="Example Combined Real & Fake Image", interactive=False, value='/tmp/example_combined_image.jpg')

    gr.Markdown(
        """
        This section visually demonstrates the robustness of our deepfake detection system.
        """
    )

    gr.Markdown("---")

    # Navigation Button
    navigate_btn = gr.Button("➡️ Navigate to the 'Detect' tab to run predictions", variant="primary")
    navigate_btn.click(lambda: gr.Info("Please click on the '🔍 Detect' tab at the top of the page to start deepfake detection."), inputs=[], outputs=[])



with gr.Blocks(title="Deep Fake Identification") as detect_page:
    gr.Markdown("""
    # 🔍 DeepFake Detector
    Select a model, choose the file type, upload your file, and click **Submit**.
    """)

    with gr.Row():
        model_choice = gr.Dropdown(
            ["ViT Model", "ResNet Model"],
            value="ViT Model",
            label="Select Model"
        )
        filetype_choice = gr.Dropdown(
            ["Image"],
            value="Image",
            label="Select File Type"
        )

    with gr.Row():
        img_input = gr.Image(type="pil", label="Upload Image", interactive=True)

    submit_btn = gr.Button("Submit")

    output_label = gr.Textbox(label="Prediction")
    output_conf = gr.Textbox(label="Confidence Score")
    output_image = gr.Image(label="Uploaded / Processed Image")

    submit_btn.click(
    None,
    inputs=[model_choice, filetype_choice, img_input],
    outputs=[output_label, output_conf, output_image]
)


# --------------------------------------------------
# COMBINED INTERFACE WITH TABS (PAGES)
# --------------------------------------------------
app = gr.TabbedInterface(
    interface_list=[home_page, detect_page],
    tab_names=["🏠 Home", "🔍 Detect"]
)

app.launch()